#  Limpeza e adequação da nomenclatura 

## Entrada dos dados

### Preparando ambiente

In [5]:
#importando bibliotecas 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### Importando dados de julho de 2022

In [ ]:
dados_brutos_julho = pd.read_csv('/content/precos-gasolina-etanol-julho.csv', sep=';')
dados_brutos_julho.head()

##### ou

In [ ]:
dados_brutos_julho = "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsan/2022/precos-gasolina-etanol-07.csv"
jul22 = pd.read_csv(dados_brutos_julho, sep = ';')
jul22.head(4)

### Importando dados de agosto de 2022

In [ ]:
dados_brutos_agosto = pd.read_csv('/content/precos-gasolina-etanol-agosto.csv', sep=';')
dados_brutos_agosto.head()

##### ou

In [ ]:
dados_brutos_agosto= "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsan/2022/precos-gasolina-etanol-08.csv"
ago22 = pd.read_csv(dados_brutos_agosto, sep=';')
ago22.head(4)

## União, adequação de dados e de nomenclatura

### Modificação do padrão das datas

In [ ]:
#dados de julho
dados_brutos_julho['Data da Coleta'] = pd.to_datetime(dados_brutos_julho['Data da Coleta'], dayfirst=True)
dados_brutos_julho['Data da Coleta'].head()

In [ ]:
#dados de agosto
dados_brutos_agosto['Data da Coleta'] = pd.to_datetime(dados_brutos_agosto['Data da Coleta'], dayfirst=True)
dados_brutos_agosto['Data da Coleta'].head()

### União dos dataframes 

In [ ]:
lista_dados = [dados_brutos_julho ,dados_brutos_agosto]
dados_brutos = pd.concat(lista_dados, ignore_index= True, verify_integrity=True)

### Momenclatura das colunas 

In [ ]:
#modificacao do nome de columas
dados_brutos.columns = dados_brutos.columns.str.replace('- ','')
dados_brutos.columns = dados_brutos.columns.str.replace(' ','_')
dados_brutos.columns = dados_brutos.columns.str.lower()
dados_brutos.columns

## Verificação de valores 

### Adequação dos valores da coluna valor_de_venda

In [ ]:
#trocando vigula por ponto e nudando de string para float
valores_virgula = dados_brutos['valor_de_venda'].tolist()
valores_ponto = [x.replace(',', '.') for x in valores_virgula]
dados_brutos['valor_de_venda'] = valores_ponto
dados_brutos['valor_de_venda'] = dados_brutos['valor_de_venda'].astype(float)

In [ ]:
dados_brutos.info()

### Verificação colunas

In [ ]:
dados_brutos.info()

Aparentemente 'valor_de_compra' esta vazia e poucos valores em 'complemento'

In [ ]:
#verificando se a coluna esta vazia 
dados_brutos.valor_de_compra.unique()

In [ ]:
dados_brutos.drop(columns=['valor_de_compra'], inplace=True)

In [ ]:
dados_brutos.info()

verificando coluna complemento

In [ ]:
dados_brutos.complemento.unique()

In [ ]:
dados_brutos.drop(columns=['complemento'], inplace=True)

## Verificando valores destoantes 

In [ ]:
dados_brutos.valor_de_venda.describe()

In [ ]:
dados_brutos.valor_de_venda.plot(kind='hist')

In [ ]:
sns.boxplot(dados_brutos.valor_de_venda)

### Verificando produtos 

In [ ]:
dados_brutos.produto.unique()

Gasolina

In [ ]:
dados_brutos.query("produto == 'GASOLINA'").describe()

In [ ]:
sns.boxplot(dados_brutos.query("produto == 'GASOLINA'").valor_de_venda)

In [ ]:
dados_brutos.query("valor_de_venda > 7").estado_sigla.unique()

# Salvando e verificando arquivo salvo

## Salvando em csv

In [ ]:
dados_brutos.to_csv('dados\dados_limpos\precos_gasolina_etanol_julho_agosto.csv', sep=',', index=False)

## Verificando arquivo salvo

In [ ]:
dados_novo_arquivo = pd.read_csv('dados\dados_limpos\precos_gasolina_etanol_julho_agosto.csv')
dados_novo_arquivo.head()

In [ ]:
dados_novo_arquivo.info()

In [ ]:
#descrever as siglas das regiões para melhor visualização das queries
dados_novo_arquivo.regiao_sigla= dados_brutos.regiao_sigla.map({'CO':'CO: Centro-Oeste', 'SE': 'SE: Sudeste', 'S':'S: Sul', 'NE': 'NE: Nordeste', 'N': 'N: Norte'})
dados_novo_arquivo.regiao_sigla

* ### Pergunta 1

In [ ]:
venda_por_data_gasolina_aditivada = dados_brutos.loc[dados_brutos['produto'] == 'GASOLINA ADITIVADA'].groupby(['data_da_coleta'])['valor_de_venda'].mean().reset_index()
venda_por_data_gasolina = dados_brutos.loc[dados_brutos['produto'] == 'GASOLINA'].groupby(['data_da_coleta'])['valor_de_venda'].mean().reset_index()
venda_por_data_etanol = dados_brutos.loc[dados_brutos['produto'] == 'ETANOL'].groupby(['data_da_coleta'])['valor_de_venda'].mean().reset_index()
venda_por_data = dados_brutos.groupby(['data_da_coleta'])['valor_de_venda'].mean().reset_index()
venda_por_data.head(10)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(venda_por_data['data_da_coleta'], venda_por_data['valor_de_venda'], label='Combustíveis') 
plt.plot(venda_por_data_etanol['data_da_coleta'], venda_por_data_etanol['valor_de_venda'], label='Etanol', linestyle="--")
plt.plot(venda_por_data_gasolina['data_da_coleta'], venda_por_data_gasolina['valor_de_venda'], label='Gasolina', linestyle="-.")
plt.plot(venda_por_data_gasolina_aditivada['data_da_coleta'], venda_por_data_gasolina_aditivada['valor_de_venda'], label='Gasolina Aditivada', linestyle=":")
plt.xlabel("X-axis")  
plt.ylabel("Y-axis")  
plt.title("Evolução do preço dos combustíveis")  
plt.legend()
plt.show() 

* ### Pergunta 2

preço médio da gasolina nos dois meses:

In [ ]:
dados_novo_arquivo.loc[dados_novo_arquivo['produto'] == 'GASOLINA']['valor_de_venda'].mean()

preço médio do etanol nos dois meses:

In [ ]:
dados_novo_arquivo.loc[dados_novo_arquivo['produto'] == 'ETANOL']['valor_de_venda'].mean()

### Pergunta 3 & 4


In [ ]:
# Quais os 5 estados com preço médio mais alto?

#1. conferindo DF:

dados_uteis=dados_novo_arquivo[['estado_sigla','produto','revenda','cnpj_da_revenda','valor_de_venda']]

display(dados_uteis)

In [ ]:
#2. Filtrando valores:

#Tirando a média da Gasolina e do Etanol
media_gas=dados_uteis.loc[dados_uteis['produto'] == 'GASOLINA']['valor_de_venda'].mean()
media_et=dados_uteis.loc[dados_uteis['produto'] == 'ETANOL']['valor_de_venda'].mean()


In [ ]:
# 3. Resgatando valores acima da média para Gasolina

gas_df=dados_uteis.loc[dados_uteis['produto'] == 'GASOLINA']
mediagas_df= gas_df.loc[gas_df['valor_de_venda'] >= media_gas]
mediagas_df.head()                     


In [ ]:
# 4. Resgatando valores acima da média para Etanol
eta_df=dados_uteis.loc[dados_uteis['produto'] == 'ETANOL']
mediaeta_df= eta_df.loc[eta_df['valor_de_venda'] >= media_et]
mediaeta_df.head()  

In [ ]:
# 5.  Preço médio da Gasolina por estado
gas_df.groupby(['estado_sigla']).mean()

In [ ]:
# 6.  Preço médio do Etanol por estado
eta_df.groupby(['estado_sigla']).mean()

In [ ]:
# 7. Qual a região que possui o maior valor médio da gasolina? 
regiao_maior_med_gas = dados_novo_arquivo[dados_novo_arquivo['produto']=='GASOLINA'].groupby(['regiao_sigla', 'produto']).aggregate( {"valor_de_venda":"mean"}).round(3).sort_values('valor_de_venda', ascending=False)
regiao_maior_med_gas.style.background_gradient(cmap='Oranges')

In [ ]:
regiao_maior_med_gas.max()

In [ ]:
# 8. Qual a região que possui o menor valor médio do etanol?
regiao_menor_med_etanol = dados_novo_arquivo[dados_novo_arquivo['produto']=='ETANOL'].groupby(['regiao_sigla', 'produto']).aggregate( {"valor_de_venda":"mean"}).round(3).sort_values('valor_de_venda')
regiao_menor_med_etanol.style.background_gradient(cmap='BuPu')


In [ ]:
regiao_menor_med_etanol.min()

In [ ]:
vl_medio_regiao = dados_brutos.groupby(['regiao_sigla', 'produto'])['valor_de_venda'].mean().reset_index()

In [ ]:
grafico_val_med_reg = plt.figure(figsize=(60,30))
plt.title('PREÇO MÉDIO - GASOLINA COMUM x ETANOL x GASOLINA ADITIVADA', fontsize=40)
sns.set(font_scale=3)
sns.barplot(y='valor_de_venda', x='regiao_sigla', hue='produto',  data = vl_medio_regiao, palette='plasma');
plt.ylabel('VALOR MÉDIO DA GASOLINA')
plt.xlabel('REGIÃO')
plt.legend(title="PRODUTO")
plt.tight_layout()